In [ ]:
import warnings
warnings.filterwarnings('ignore')
from nsepython import *
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta 
from nselib import capital_market
from datetime import date
import yfinance as yf
import panel as pn
pn.extension()
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows',None)
start_date = (date.today() - relativedelta(years=2)).strftime('%d-%m-%Y')
end_date = (date.today() - relativedelta(days=1)).strftime('%d-%m-%Y')
start_date_yf = (date.today() - relativedelta(years=2)).strftime('%Y-%m-%d')
end_date_yf = (date.today() - relativedelta(days=1)).strftime('%Y-%m-%d')

series = "EQ"
def stock_fetcher():
    symbol_df = capital_market.equity_list()            # library to get data of symbols
    symbol_df.columns = symbol_df.columns.str.strip()
    return list(symbol_df['SYMBOL'].unique())
input_symbol = pn.widgets.Select(name='SYMBOLS',options=stock_fetcher()).servable(area='sidebar') # widget to choose symbol
input_symbol

In [ ]:
# fetching historical data using nsepython

symbol = input_symbol.value
data = equity_history(symbol,series,start_date,end_date)
data = data[['CH_SYMBOL','CH_SERIES','CH_TIMESTAMP','CH_OPENING_PRICE','CH_TRADE_HIGH_PRICE','CH_TRADE_LOW_PRICE','CH_CLOSING_PRICE','CH_LAST_TRADED_PRICE','CA']]
data.rename(columns={'CH_TIMESTAMP':'date','CH_SYMBOL':'symbol','CH_SERIES':'Series','CH_OPENING_PRICE':'Open','CH_TRADE_HIGH_PRICE':'High','CH_TRADE_LOW_PRICE':'Low','CH_CLOSING_PRICE':'Close','CH_LAST_TRADED_PRICE':'Ltp'},inplace=True)
data = data.drop_duplicates(subset=['date']).sort_values(by=['date'], ascending = False).reset_index(drop=True)
data_ca = data[~data['CA'].isna()].reset_index(drop=True)
df_ex = pd.json_normalize(data_ca['CA'].str[0])
data_of_corp = pd.concat([data_ca.drop(columns=['CA']), df_ex], axis=1)
face_value = float(data_of_corp['faceVal'].unique())
corporate_action = dict(zip(data_of_corp['date'],data_of_corp['subject']))
print(corporate_action)

In [ ]:
### Calculates the Adjuted Close

my_dict_for_corpo = {}
data["Divi_Multiplier"] = 1
data["Split_Multiplier"] = 1
data["bonus_Multiplier"] = 1
data["Right_Multiplier"] = 1
for date,action in corporate_action.items():

    ## Splits
    
    if 'Split' in action:
        data_li = (action.split('From '))[1].split(' ')
        numerical_data = [re.search(r'\d+', item) for item in data_li]
        numerical_values = [int(match.group()) for match in numerical_data if match]
        ratio = numerical_values[1]/numerical_values[0]
        my_dict_for_corpo[date] = {'Split' : ratio}
        data.loc[data['date'] == date, 'split'] = ratio
        data['split'] = data['split'].fillna(1)
        for i in range(1, len(data)):
            data.at[i, "Split_Multiplier"] = data.at[i-1, "Split_Multiplier"] * data.at[i-1, "split"]

    ## Bonus
    
    if 'Bonus' in action:
        Bonus_list = action.split(' ')[1].split(':')
        bonus = int(Bonus_list[1]) / (int(Bonus_list[0]) + int(Bonus_list[1]))
        my_dict_for_corpo[date] = {'Bonus': bonus}
        data.loc[data['date'] == date, 'bonus'] = bonus
        data['bonus'] = data['bonus'].fillna(1)
        for i in range(1, len(data)):
            data.at[i, "bonus_Multiplier"] = data.at[i-1, "bonus_Multiplier"] * data.at[i-1, "bonus"]

    ## Right Issue
    
    if 'Rights' in action:
        print(date)
        ratio = (action.split('@')[0].split('Rights')[1].split(':'))
        premium = float(action.split('@')[1].split('Rs ')[1].split('/')[0])
        S =  face_value + premium
        P = float(data.loc[data['date'] == date, 'Close'].iloc[-1])
        A = float(ratio[0])
        B = float(ratio[1])
        C = (P - S) * A
        E = C / (A + B)
        AF = (P - E)/ P
        print(AF)
        my_dict_for_corpo[date] = {'Rights': f'{ratio}@{S}'}
        data.loc[data['date'] == date, 'Right'] = AF
        data['Right'] = data['Right'].fillna(1)
        for i in range(1, len(data)):
            data.at[i, "Right_Multiplier"] = data.at[i-1, "Right_Multiplier"] * data.at[i-1, "Right"]

    ## Dividend
    
    if 'Dividend' in action:
        action = action.replace("Re", "Rs").replace("Rs.", "Rs")
        my_dict_for_corpo[date] = {'Dividend':float(action.split('Rs ')[1].split('Per')[0])}
        data.loc[data['date'] == date, 'Dividend'] = float(action.split('Rs ')[1].split('Per')[0])
        data['Dividend'] = data['Dividend'].fillna(0)
        data['Divi_empact'] = (1 - (data['Dividend'] / (data["Split_Multiplier"]*data['Close'].shift(-1))))
        for i in range(1, len(data)):
            data.at[i, "Divi_Multiplier"] = data.at[i-1, "Divi_empact"] * data.at[i - 1, "Divi_Multiplier"]

data["Multiplier"] = data["Divi_Multiplier"] * data["Split_Multiplier"] * data["bonus_Multiplier"] * data["Right_Multiplier"]
data["Adj_Close"] = round(data["Close"] * data["Multiplier"], 2)

In [ ]:
data[["symbol","date","Close","Adj_Close"]]

In [ ]:
my_dict_for_corpo

In [ ]:
# Data from Yahoo Finance

req_symbol = str(input_symbol.value) + '.NS'
req_symbol
data_yf = yf.download(req_symbol, start=start_date_yf, end = end_date_yf ) 
price_data = data_yf.reset_index()#.rename(columns={'Date':'date'},inplace=True)
price_data = price_data.sort_values(by='Date',ascending=False).reset_index(drop=True)
price_data.rename(columns={'Date':'date'},inplace=True)
price_data['date'] = price_data['date'].astype(str)
price_data['Adj Close'] = round(price_data['Adj Close'], 2)

# Sanity checks:


resulted_data = pd.merge(data[['date','symbol', 'Close','Adj_Close']], price_data[['date','Adj Close']],on='date')
resulted_data.rename(columns={'Adj_Close' : 'Adj_Close_pred', 'Adj Close':'Adj_Close_Yahoo'},inplace=True)
resulted_data['diff'] = round(abs(resulted_data['Adj_Close_Yahoo'] - resulted_data['Adj_Close_pred']), 0)
resulted_data[resulted_data['diff'] != 0.0] 